In [149]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio

# Neural Networks

In [150]:
path = 'ex4data1.mat'
data = sio.loadmat(path)
X = data['X']
y = data['y']
rows = X.shape[0]
params = X.shape[1]
weight = sio.loadmat('ex4weights.mat')
theta1 = weight['Theta1']
theta2 = weight['Theta2']
y.shape

(5000, 1)

In [151]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [152]:
def sigmoidGradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))


In [153]:
def cost(theta1, theta2, X, y, num_label, alpha):
    rows = X.shape[0]
    params = X.shape[1]
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    X = np.matrix(X)
    theta1 = np.matrix(theta1)
    theta2 = np.matrix(theta2)
    A = sigmoid(X * theta1.T)
    A = np.insert(A, 0, values=np.ones(rows), axis=1)
    A = np.matrix(A)
    
    New = np.zeros((y.shape[0], 10))
    for i in range(rows):
        New[i, y[i] - 1] = 1
    y = New
    # print(A.shape, X.shape)
    # first = np.multiply(-y, np.log(sigmoid(X * theta1.T))) - np.multiply((1 - y), np.log(1 - sigmoid(X * theta1.T)))
    second =np.multiply(-y, np.log(sigmoid(A * theta2.T))) - np.multiply((1 - y), np.log(1 - sigmoid(A * theta2.T)))

    reg = np.sum(np.power(theta2, 2)) * (alpha / (2 * len(A)))
    
    return (np.sum(second)) / len(X) + reg


In [154]:
C = cost(theta1, theta2, X, y, 25, 1)
C

0.3449633357692933

# Backpropagation

In [155]:
sigmoidGradient(0)

0.25

In [156]:
epsilon_init = 0.12
L_2 = 25
L_3 = 10
theta1 = np.matrix(np.random.rand(L_2, X.shape[1] + 1) * (2 * epsilon_init) - epsilon_init)
theta2 = np.matrix(np.random.rand(L_3, L_2 + 1) * (2 * epsilon_init) - epsilon_init)
theta2.shape, theta1.shape


((10, 26), (25, 401))

In [157]:
def forwardPropagation(A, theta):
    return (A.T * theta.T).T
def insertfunction(A):
    A = np.insert(A, 0, 1, axis=0)
    return np.matrix(A)
# a_1 = np.matrix(X)
# z_2 = forwardPropagation(a_1, theta1)
# a_2 = sigmoid(z_2)
# z_3 = forwardPropagation(a_2, theta2)
# a_3 = sigmoid(z_3)
# z_2.shape

In [158]:
# New = np.zeros((y.shape[0], 10))
# for i in range(rows):
#     New[i, y[i] - 1] = 1
# y = New
# delta3 = a_3 - y
# y.shape, delta3.shape

In [177]:
def BPfunction(X, y, theta1, theta2, alpha):
    New = np.zeros((y.shape[0], 10))
    for i in range(rows):
        New[i, y[i] - 1] = 1
    y = New.T

    for i in range(len(X)):
        a1 = X[i, :].T
        y = y[:, i]
        print(y)
        #先转换成列向量！
        a1 = insertfunction(a1)
        # print(a1.shape)
        a1 = np.matrix(a1)
        z2 = forwardPropagation(a1.T, theta1) 
        print(z2.shape)
        a2 = sigmoid(z2)
        a2 = insertfunction(a2)
        z3 = forwardPropagation(a2, theta2)
        a3 = sigmoid(z3)
        print(a3.shape)
        
        delta3 = a3 - y.T
        print(delta3.shape)
        # delta2 = np.multiply(np.multiply(theta2.T, delta3.T), sigmoidGradient(z2))
        delta2 = np.multiply(theta2.T, delta3.T) * sigmoidGradient(z2)
        delta2 = delta2[1 :delta2.shape[0], :]
        


In [178]:
delta = BPfunction(X, y, theta1, theta2, 1)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
(25, 1)
(10, 1)
(10, 10)


ValueError: operands could not be broadcast together with shapes (26,10) (10,10) 